In [40]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import os
import shutil
import logging
import subprocess
from copy import deepcopy
from time import time
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdDistGeom import EmbedMultipleConfs
from linux_qm.src.util import _load_smiles3D, _create_tmp_dir, draw3Dconfs, SetPositions
from linux_qm.qm.crest.crest import conformer_pipeline


In [42]:
smi = 'COC1CN(C)C1'
# smi = 'CO'


# mol = _load_smiles3D(smi)
# EmbedMultipleConfs(mol, numConfs=3)

mol = conformer_pipeline(smi)

# Get the conformers

conformers = mol.GetConformers()

# Print the number of conformers generated
print(f"Number of conformers generated: {len(conformers)}")
print("Energies:", [c.GetProp('energy') for c in conformers])

[24-11-2023 10:32:48] [DEBUG] Success SMILES parsing
normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
[24-11-2023 10:32:48] [DEBUG] Success geometry optimization
[24-11-2023 10:32:51] [DEBUG] Success conformer generation
[24-11-2023 10:32:52] [DEBUG] Success conformer screen


Number of conformers generated: 4
Energies: ['-23.275472740000001', '-23.273513619999999', '-23.273084770000001', '-23.272812779999999']


In [43]:
query = Chem.MolFromSmarts('C1CCN1')
atom_ids = mol.GetSubstructMatch(query)
print(atom_ids)
AllChem.AlignMolConformers(mol, atomIds=atom_ids)
draw3Dconfs(mol, autoalign=False)

(3, 2, 6, 4)
num conformers 4


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [44]:
import os
os.getcwd()

'/home/ergot/linux_qm/notebooks/.orca_tmp/ad383fa1-ca2c-4edb-b6c6-25bbf7e7f286/.orca_tmp/59e6d026-c7fa-4314-a0cb-59597b561b93'

In [45]:
logging.getLogger().setLevel(logging.DEBUG)

In [46]:
from linux_qm.qm.orca.orca import OrcaDriver
orca = OrcaDriver()

conf = deepcopy(mol).GetConformer()

orca.options['method'] = 'HF def2-SVP'
orca.single_point(conf)

[24-11-2023 10:32:52] [DEBUG] ORCA INPUT:
!HF def2-SVP   
%geom MaxIter 100 end
%pal nprocs 16 end
%maxcore 2000
*xyz 0 1
C            2.71485338         -0.42879529         -0.15402121
O            1.61469872          0.39692838         -0.44575377
C            0.56782254          0.27290454          0.47463673
C           -0.36189580         -0.95006192          0.33946601
N           -1.51499837         -0.04690265          0.36976633
C           -2.58129655         -0.29499503         -0.56442482
C           -0.66124272          1.11989988          0.13118964
H            3.46306861         -0.22800980         -0.91805051
H            3.12948907         -0.19811282          0.83512158
H            2.43980969         -1.48947411         -0.18098431
H            0.93020891          0.41944030          1.50068762
H           -0.36203289         -1.68046678          1.14985881
H           -0.22259949         -1.45659519         -0.62370349
H           -3.06203357         -1.23783845   

In [47]:
orca.geometry_optimization(conf)

[24-11-2023 10:32:56] [DEBUG] ORCA INPUT:
!HF def2-SVP OPT  
%geom MaxIter 100 end
%pal nprocs 16 end
%maxcore 2000
*xyz 0 1
C            2.71485338         -0.42879529         -0.15402121
O            1.61469872          0.39692838         -0.44575377
C            0.56782254          0.27290454          0.47463673
C           -0.36189580         -0.95006192          0.33946601
N           -1.51499837         -0.04690265          0.36976633
C           -2.58129655         -0.29499503         -0.56442482
C           -0.66124272          1.11989988          0.13118964
H            3.46306861         -0.22800980         -0.91805051
H            3.12948907         -0.19811282          0.83512158
H            2.43980969         -1.48947411         -0.18098431
H            0.93020891          0.41944030          1.50068762
H           -0.36203289         -1.68046678          1.14985881
H           -0.22259949         -1.45659519         -0.62370349
H           -3.06203357         -1.23783845

In [48]:
%%time

logging.getLogger().setLevel(logging.INFO)
orca.options['solvent'] = 'THF'


conf = deepcopy(mol).GetConformer()

orca.options['method'] = 'M062X 6-311++G(d,p) NMR'
orca.geometry_optimization(conf)

[24-11-2023 10:35:40] [INFO] Identified logfile to be in ORCA format
[24-11-2023 10:35:40] [INFO] Success: True
[24-11-2023 10:35:40] [INFO] Num Iter: 9


CPU times: user 120 ms, sys: 64.7 ms, total: 185 ms
Wall time: 2min 11s


In [49]:
%%time

conf = deepcopy(mol).GetConformer()

orca.options['method'] = 'M062X 6-31G'
orca.geometry_optimization(conf)

orca.options['method'] = 'M062X 6-311++G(d,p) NMR'
orca.geometry_optimization(conf)

[24-11-2023 10:36:32] [INFO] Identified logfile to be in ORCA format
[24-11-2023 10:36:32] [INFO] Success: True
[24-11-2023 10:36:32] [INFO] Num Iter: 9
[24-11-2023 10:38:15] [INFO] Identified logfile to be in ORCA format
[24-11-2023 10:38:15] [INFO] Success: True
[24-11-2023 10:38:15] [INFO] Num Iter: 7


CPU times: user 231 ms, sys: 55.6 ms, total: 286 ms
Wall time: 2min 34s


In [44]:
%%time

conf = deepcopy(mol).GetConformer()
options['method'] = 'M062X def2-TZVP OPT'
data, output = run_orca(conf, options)

[23-11-2023 08:08:50] [INFO] Identified logfile to be in ORCA format


Success: True
Elapsed time: 0:02:26.726000
Num iter: 9
CPU times: user 141 ms, sys: 31 ms, total: 172 ms
Wall time: 2min 26s


In [37]:
%%time

conf = deepcopy(mol).GetConformer()
options['method'] = 'M062X aug-cc-pVDZ OPT'
data, output = run_orca(conf, options)

[23-11-2023 07:40:13] [INFO] Identified logfile to be in ORCA format


Success: True
Elapsed time: 0:02:58.253000
Num iter: 9
CPU times: user 143 ms, sys: 32.7 ms, total: 176 ms
Wall time: 2min 58s


In [41]:
%%time

conf = deepcopy(mol).GetConformer()
options['method'] = 'M062X cc-pVTZ OPT'
data, output = run_orca(conf, options)

[23-11-2023 07:54:31] [INFO] Identified logfile to be in ORCA format


Success: True
Elapsed time: 0:03:20.218000
Num iter: 9
CPU times: user 129 ms, sys: 66.7 ms, total: 196 ms
Wall time: 3min 20s


In [42]:
%%time

conf = deepcopy(mol).GetConformer()
options['method'] = 'M062X aug-cc-pVTZ OPT'
data, output = run_orca(conf, options)

[23-11-2023 08:03:34] [INFO] Identified logfile to be in ORCA format


Success: True
Elapsed time: 0:09:02.365000
Num iter: 9
CPU times: user 160 ms, sys: 83.4 ms, total: 243 ms
Wall time: 9min 2s


In [45]:
%%time

conf = deepcopy(mol).GetConformer()
options['method'] = 'M062X cc-pVDZ OPT'
data, output = run_orca(conf, options)

options['method'] = 'M062X aug-cc-pVTZ OPT NMR'
data, output = run_orca(conf, options)

[23-11-2023 08:30:21] [INFO] Identified logfile to be in ORCA format


Success: True
Elapsed time: 0:01:21.496000
Num iter: 9


[23-11-2023 08:35:16] [INFO] Identified logfile to be in ORCA format


Success: True
Elapsed time: 0:04:55.687000
Num iter: 5
CPU times: user 215 ms, sys: 92.4 ms, total: 308 ms
Wall time: 6min 17s


In [43]:
query = Chem.MolFromSmarts('C1CCN1')
atom_ids = mol.GetSubstructMatch(query)
print(atom_ids)
AllChem.AlignMolConformers(mol, atomIds=atom_ids)
draw3Dconfs(mol, confIds=[0], autoalign=False)

(3, 2, 6, 4)
num conformers 4


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [52]:
data.scfvalues

[array([[ 0.00000000e+00,  1.75780800e-01,  1.13529000e-03],
        [-7.83250489e-02,  1.44294260e-01,  9.00510000e-04],
        [-3.82209220e-02,  3.60672850e-01,  2.12364000e-03],
        [-5.95070839e-02,  3.45931600e-02,  3.53380000e-04],
        [-1.10323800e-02,  1.38156100e-02,  1.32650000e-04],
        [-2.43487796e-04,  4.96315000e-03,  5.04600000e-05],
        [-3.80466000e-05,  2.50000000e-04,  7.07300000e-03],
        [-5.39920000e-06,  1.49000000e-04,  3.84100000e-03],
        [-3.79500000e-07,  6.80000000e-05,  8.82000000e-04],
        [-4.23500000e-07,  1.50000000e-05,  6.69000000e-04],
        [-8.50000000e-09,  7.00000000e-06,  7.70000000e-05],
        [-5.17330000e-09,  3.99680000e-15,  1.17560000e-17]]),
 array([[-3.27054152e+02,  2.26400000e-03,  6.88800000e-03],
        [-3.97856500e-04,  4.46000000e-04,  5.24900000e-03],
        [-3.05938000e-05,  2.93000000e-04,  2.26000000e-03],
        [-1.46754000e-05,  6.50000000e-05,  7.32000000e-04],
        [-2.08900000e-

In [51]:
data.scftargets

array([[1.e-08, 1.e-07, 5.e-09],
       [1.e-08, 1.e-07, 5.e-09],
       [1.e-08, 1.e-07, 5.e-09],
       [1.e-08, 1.e-07, 5.e-09],
       [1.e-08, 1.e-07, 5.e-09]])

In [36]:
# NMR shielding
for k,v in data.nmrtensors.items():
    print(mol.GetAtomWithIdx(k).GetSymbol(), k, v['isotropic'])

C 0 125.508
O 1 275.301
C 2 110.446
C 3 114.12
N 4 223.565
C 5 133.679
C 6 115.657
H 7 28.766
H 8 28.318
H 9 28.777
H 10 27.965
H 11 28.106
H 12 29.335
H 13 29.221
H 14 30.004
H 15 29.269
H 16 29.104
H 17 28.096


In [15]:
dir(data)

['OPT_DONE',
 'OPT_NEW',
 'OPT_UNCONVERGED',
 'OPT_UNKNOWN',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attributes',
 '_attrlist',
 '_dictsofarrays',
 '_intarrays',
 '_listsofarrays',
 'arrayify',
 'atomcharges',
 'atomcoords',
 'atommasses',
 'atomnos',
 'charge',
 'check_values',
 'closed_shell',
 'converged_geometries',
 'coreelectrons',
 'geotargets',
 'geovalues',
 'getattributes',
 'grads',
 'homos',
 'listify',
 'metadata',
 'moenergies',
 'moments',
 'mosyms',
 'mult',
 'natom',
 'nbasis',
 'nelectrons',
 'new_geometries',
 'nmo',
 'nmrtensors',
 'optdone',
 'scfenergies',
 'scftargets',
 'scfvalues',
 'setattributes',
 'typecheck',
 'unconverged_geometr

In [16]:
print(data.metadata['cpu_time'][0])

0:01:17.614000


In [17]:
len(data.converged_geometries)

5

In [18]:
dir(data)

['OPT_DONE',
 'OPT_NEW',
 'OPT_UNCONVERGED',
 'OPT_UNKNOWN',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attributes',
 '_attrlist',
 '_dictsofarrays',
 '_intarrays',
 '_listsofarrays',
 'arrayify',
 'atomcharges',
 'atomcoords',
 'atommasses',
 'atomnos',
 'charge',
 'check_values',
 'closed_shell',
 'converged_geometries',
 'coreelectrons',
 'geotargets',
 'geovalues',
 'getattributes',
 'grads',
 'homos',
 'listify',
 'metadata',
 'moenergies',
 'moments',
 'mosyms',
 'mult',
 'natom',
 'nbasis',
 'nelectrons',
 'new_geometries',
 'nmo',
 'nmrtensors',
 'optdone',
 'scfenergies',
 'scftargets',
 'scfvalues',
 'setattributes',
 'typecheck',
 'unconverged_geometr

In [19]:
data.optdone

True

In [20]:
data.metadata

{'package': 'ORCA',
 'methods': ['DFT', 'DFT', 'DFT', 'DFT', 'DFT'],
 'success': True,
 'legacy_package_version': '5.0.3',
 'package_version': '5.0.3',
 'basis_set': '6-311++G(d,p)',
 'warnings': ['Geometry OptimizationSwitching off AutoStartFor restart on a previous wavefunction, please use MOREAD',
  'Minnesota functionals are quite sensitive to the integration grid.see SE Wheeler, KN Houk, JCTC 2010, 6, 395,N Mardirossian, M Head-Gordon, JCTC 2016, 12, 4303.DEFGRID3 seems to be a minimum grid for reliable results with these functionals!Please increase the integration grid!'],
 'info': ['the flag for use of the SHARK integral package has been found!'],
 'input_file_name': 'task.inp',
 'input_file_contents': '!M062X 6-311++G(d,p) OPT NMR\n%geom MaxIter 100 end\n%cpcm\n  smd true\n  SMDsolvent "THF"\n  end\n%pal nprocs 16 end\n*xyz 0 1\nC           -2.73166000         -0.42891300          0.06568100\nO           -1.69258800          0.47020300          0.38413500\nC           -0.584961

In [21]:
data.new_geometries

array([[[-2.73166 , -0.428913,  0.065681],
        [-1.692588,  0.470203,  0.384135],
        [-0.584961,  0.294699, -0.444423],
        [ 0.318446, -0.937096, -0.217537],
        [ 1.481163, -0.047712, -0.386694],
        [ 2.63364 , -0.299075,  0.451037],
        [ 0.645131,  1.118235, -0.052189],
        [-3.11054 , -0.260819, -0.952362],
        [-3.541348, -0.262969,  0.778925],
        [-2.400831, -1.474237,  0.143859],
        [-0.856075,  0.412823, -1.502824],
        [ 0.270658, -1.784797, -0.907525],
        [ 0.201829, -1.289001,  0.821651],
        [ 3.376249,  0.492224,  0.308155],
        [ 2.374666, -0.342195,  1.523777],
        [ 3.093524, -1.251502,  0.169314],
        [ 0.615582,  1.32106 ,  1.030892],
        [ 0.87263 ,  2.037938, -0.598603]],

       [[-2.732772, -0.433447,  0.065903],
        [-1.697911,  0.474342,  0.37912 ],
        [-0.585376,  0.300297, -0.445689],
        [ 0.315233, -0.934156, -0.217946],
        [ 1.481057, -0.046769, -0.385799],
        [

In [22]:
# HOMO LUMO
homo = data.moenergies[0][data.homos[0]]
lumo = data.moenergies[0][data.homos[0] + 1]
homo, lumo

(-7.7628095042939, 0.14092776317395)

In [23]:
print(data.metadata['cpu_time'][0])

0:01:17.614000


In [24]:
data.natom, data.nmo

(18, 231)

In [25]:
data.metadata

{'package': 'ORCA',
 'methods': ['DFT', 'DFT', 'DFT', 'DFT', 'DFT'],
 'success': True,
 'legacy_package_version': '5.0.3',
 'package_version': '5.0.3',
 'basis_set': '6-311++G(d,p)',
 'warnings': ['Geometry OptimizationSwitching off AutoStartFor restart on a previous wavefunction, please use MOREAD',
  'Minnesota functionals are quite sensitive to the integration grid.see SE Wheeler, KN Houk, JCTC 2010, 6, 395,N Mardirossian, M Head-Gordon, JCTC 2016, 12, 4303.DEFGRID3 seems to be a minimum grid for reliable results with these functionals!Please increase the integration grid!'],
 'info': ['the flag for use of the SHARK integral package has been found!'],
 'input_file_name': 'task.inp',
 'input_file_contents': '!M062X 6-311++G(d,p) OPT NMR\n%geom MaxIter 100 end\n%cpcm\n  smd true\n  SMDsolvent "THF"\n  end\n%pal nprocs 16 end\n*xyz 0 1\nC           -2.73166000         -0.42891300          0.06568100\nO           -1.69258800          0.47020300          0.38413500\nC           -0.584961

In [26]:
data.scfenergies

array([-8898.73503472, -8898.73716429, -8898.73758198, -8898.73766933,
       -8898.73770797])